# TODO

* find the varid horizon

In [1]:
import gym, pickle
from itertools import product
from copy import deepcopy
import tensorflow as tf
import ray
from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.evaluation import PolicyGraph, PolicyEvaluator, SampleBatch
from ray.rllib.evaluation.metrics import collect_metrics
from ray.tune.registry import register_env

from flow.multiagent_envs import MultiWaveAttenuationPOEnv
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder, get_flow_params

In [16]:
num_cpus = 3
num_rollouts = 3
horizon = 750
gae_lambda = 0.97
step_size = 5e-4
num_iter = 10
benchmark_name = "multi_merge"

In [4]:
ray.init(num_cpus=num_cpus, logging_level=40, ignore_reinit_error=True)

{'node_ip_address': '169.237.32.118',
 'object_store_address': '/tmp/ray/session_2019-05-24_21-44-07_1927/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-24_21-44-07_1927/sockets/raylet',
 'redis_address': '169.237.32.118:45662',
 'webui_url': None}

In [17]:
config = deepcopy(DEFAULT_CONFIG)
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["sample_batch_size"] = horizon
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [128, 64, 32]
config["observation_filter"] = "NoFilter"
config["entropy_coeff"] = 0.0
config["num_iter"] = num_iter

In [15]:
benchmark = __import__(
            "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.buffered_obs_flow_params
create_env, env_name = make_create_env(params=flow_params, version=0)
register_env(env_name, create_env)
env = create_env()

In [18]:
def training_workflow(config, reporter):
    env = create_env()   
    policy = PPOPolicyGraph(env.observation_space, env.action_space, config)
    workers = [
        PolicyEvaluator.as_remote().remote(
            create_env, 
            policy_graph=policy_graph,
            policy_mapping_fn=policy_mapping_fn,
            batch_steps=config["sample_batch_size"])
        for _ in range(config["num_workers"])]
    
    for _ in range(config["num_iter"]):
        weights = ray.put({"default_policy": policy.get_weights()})
        for w in workers:
            w.set_weights.remote(weights)       
        batch = SampleBatch.concat_samples(
            ray.get([w.sample.remote() for w in workers]))
        policy.learn_on_batch(batch)
        reporter(**collect_metrics(remote_evaluators=workers))

def main():
    default_policy = (PPOPolicyGraph, env.observation_space, env.action_space, config)
    policy_graph={"default_policy": default_policy}
    policy_mapping_fn=lambda agent_id: "default_policy"
    
    tune.run(
        training_workflow,
        config=config
    )

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

/opt/conda/envs/flow-latest/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
